In [11]:
import import_ipynb
import sys
import torch
from torch import nn
from torch import optim                                        

if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

%cd /home/ankit/Commitments/earthquake-prediction/

from preprocessing import *
from training import *
from postprocessing import *
from testing import *

%cd /home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi/

/home/ankit/Commitments/earthquake-prediction
/home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi


In [19]:
# SETTINGS D

data_file = "data.csv"
time_parser = time_from_string_dmy
mag_range = (5, 6)
params_file = "light-cnn-lstm.params"
rows = 3
cols = 3
total_params = rows*cols
time_intervals = 690
train_size = 0.8
lookback_window = 50

In [20]:
earthquakes = extract_from_csv(data_file, time_parser, mag_range)
cells = Spatio_Temporal_Segmentation(rows, cols, time_intervals, earthquakes)

print(cells[0][0][0])

No earthquakes were recorded in this cell.


In [21]:
generator = TensorGen(cells)

eventtensor = generator.eventNoEvent()
# eventtensor = eventtensor.long()
print(eventtensor.dtype)
freqtensor = generator.freq()
minmagtensor = generator.min_magnitude()
maxmagtensor = generator.max_magnitude()

if torch.cuda.is_available:
    eventtensor = eventtensor.cuda()
    freqtensor = freqtensor.cuda()
    minmagtensor = minmagtensor.cuda()
    maxmagtensor = maxmagtensor.cuda()
    
print(eventtensor.size())
print(freqtensor.size())
print(minmagtensor.size())
print(maxmagtensor.size())

torch.float32
torch.Size([690, 1, 9])
torch.Size([690, 1, 9])
torch.Size([690, 1, 9])
torch.Size([690, 1, 9])


<strong>TRAINING EVENT PREDICTOR</strong>

In [22]:
#SETTINGS M1

hidden_parameters = 200
linear_parameters = 50
num_layers = 1

In [23]:

model = CNN_LSTM(in_channels=lookback_window, out_channels=80, kernel_size=3, in_parameters=rows*cols, hidden_parameters=hidden_parameters, num_layers=num_layers, activation=nn.Tanh())
# model.load_state_dict(torch.load(params_file))
if torch.cuda.is_available():
    model = model.cuda()
    
# out = model(eventtensor[:10])
# print(out)

In [24]:
# SETTINGS T1
pos_weight=torch.tensor([1])
# weight = None

# print(weight)
pos_weight = pos_weight.cuda()
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
n_epochs = 100

In [25]:
optimizer = optim.Adagrad(model.parameters(), lr=0.01)

# tqdm._instances.clear()
train(model, eventtensor, optimizer, loss_function, n_epochs=5)

  2%|▏         | 12/640 [00:00<00:05, 109.59it/s]

Epoch 1 completed. 
Training Loss: 0.568
Test Loss: 0.577


  2%|▏         | 12/640 [00:00<00:05, 111.52it/s]

Epoch 2 completed. 
Training Loss: 0.564
Test Loss: 0.578


 17%|█▋        | 108/640 [00:02<00:10, 50.98it/s]


KeyboardInterrupt: 

In [ ]:
# params_file = "light-cnn-lstm.params"
# torch.save(model.state_dict(), params_file)

In [26]:
evaluate(data=eventtensor, model=model, train_size=0.8, lookback_window=lookback_window, threshold=0.5)

100%|██████████| 640/640 [00:02<00:00, 227.94it/s]


Evaluation results for training dataset:
4518 evaluations in total.
834/1971 cells with earthquake activity were correctly identified. (42.31 %)
2377/2547 cells with no earthquake activity were correctly identified. (93.33 %)
170/1004 positives turned out to be false.
1137/3514 negatives turned out to be false.
Preicision: 0.831
Recall: 0.423
Overall accuracy: 71.07 %
Evaluation results for training dataset:
1242 evaluations in total.
206/493 cells with earthquake activity were correctly identified. (41.78 %)
679/749 cells with no earthquake activity were correctly identified. (90.65 %)
70/276 positives turned out to be false.
287/966 negatives turned out to be false.
Preicision: 0.746
Recall: 0.418
Overall accuracy: 71.26 %
